<a href="https://colab.research.google.com/github/JJLimmm/optimizationresearch/blob/main/OR_Tools_LP_Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operations Optimization II: Examples for Google OR-Tools

Here are some practice examples for the Operations Optimization II lessons on _**Simplex Method**_. 

You should finish the installation of **Google OR-Tools** and **Python** in your environment before starting on this practice code. The installation of Google OR-Tools can be found [here](https://developers.google.com/optimization/install)

Below are examples for Linear Programming (**LP**) cases in order to help you understand how to solve optimization programs with the OR-Tools library.

==================================================================================================================

In [ ]:
# First, we will install the libraries required for our examples
!python -m pip install -U ortools
!pip install pandas
!pip install prettytable

## Practice 1: Producing desks and tables

Consider the problem we introduced in Operations Optimization II, we have the LP problem where...

\begin{split}
    x_1 = \mbox{ number of desks produced in a day } \\
    x_2 = \mbox{ number of tables produced in a day }
\end{split}

\begin{split}
    \begin{array}{r}
        \max \\ \mbox{s.t.} \\ \\ \\ \\ \\ 
    \end{array} &
    \begin{array}{rcrcll}
        700x_1 & + & 900x_2 & & & \\ 
        3x_1 & + & 5x_2 & \leq & 3600\quad\! & \mbox{(wood)} \\		
        x_1 & + & 2x_2 & \leq & 1600\quad\! & \mbox{(labor)} \\		
        50x_1 & + & 20x_2 & \leq & 48000\:\ & \mbox{(machine)} \\
        & & x_1 & \geq & 0
        \\
        & & x_2 & \geq & 0.
    \end{array}
\end{split}

Let's construct our problem with Google OR-Tools step by step.



### Step 1
We should import the Google OR-Tools optimization package and its relevant libraries called *ortools* first, and then declare the **solver**.

The *pywraplp* is a Python wrapper for the underlying solver created in C++.
The [API Reference Documentation](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.OPTIMAL) can be found here.

In [ ]:
# Import libraries and packages
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

# Declare the solver
# The argument 'GLOP' is for using the Google Linear Optimization back-end package
solver = pywraplp.Solver.CreateSolver('GLOP')

### Step 2
First step would be to declare the variables being used.

In this problem, there are only 2 variables.

Let
\begin{split}
    x_1 = x \\
    x_2 = y
\end{split}

In [ ]:
# Define the variables as described above
# For different cases, you can change the values in args1 and args2 to fit the problem you have at hand.
''' 
   solver.NumVar(lb = *, ub = *, name = *)
   
   Description: Adds in Variables for the LP problem you have at hand.
   
        where...
             lb   = Lower bound for variable (Minimum value)
             ub   = Upper bound for variable (Maximum value)
             name = name for variable
'''

x = solver.NumVar(0, solver.infinity(), 'x')
y = solver.NumVar(0, solver.infinity(), 'y')

print('Number of variables =', solver.NumVariables())

### Step 3
Second step would be to define the **Objective Function**.

From the problem statement, **the Objective Function** is
\begin{split}
    \begin{array}{r}
        \max
    \end{array} &
    \begin{array}{rcrcll}
        700x_1 & + & 900x_2 & & & 
    \end{array}
\end{split}

In [ ]:
# Define the Objective Function
# Max 700x + 900y

# Initialise the solver.Objective() function from ortools library
objective = solver.Objective()

# Set Coefficients for the variables with objective.SetCoefficient(args1, args2)
"""
    objective.SetCoefficient(args1, args2)
    
    Description: Sets Coefficient of variables in the Objective function
    
        where...
            args1: variable name
            args2: coefficient value
"""
objective.SetCoefficient(x, 700)
objective.SetCoefficient(y, 900)

# Sets the optimization direction to maximize
# Alternatively, objective.SetMinimization() to minimize.
objective.SetMaximization()

### Step 4
Next step would be to declare the **constraints** in the LP Problem. \
There are **3 constraints** as seen from the problem statement

\begin{split}
    \begin{array}{r}
        \mbox{s.t.} \\ \\ \\   
    \end{array} &
    \begin{array}{rcrcll}
        3x_1 & + & 5x_2 & \leq & 3600\quad\! & \mbox{(wood)} \\		
        x_1 & + & 2x_2 & \leq & 1600\quad\! & \mbox{(labor)} \\		
        50x_1 & + & 20x_2 & \leq & 48000\:\ & \mbox{(machine)} \\
    \end{array}
\end{split}

In [ ]:
# Initialise the 3 Constraints by calling the solver.Add() function
"""
    solver.Add(args1, args2)
    
    Description: Add a constraint with the variables and you can provide a name for the constraint
    
        Input arguments:
            args1: algebraic equation for the constraint
            args2(optional): Name for the constraint\
        
        returns:
            None
"""
# Constraint 1 for wood
solver.Add( 3*x + 5*y <= 3600)

##### Add in the remaining 2 Constraints below #####
# Constraint 2 for labor

# Constraint 3 for machine


# Show the Number of constraints added into ortools
print('Number of constraints =', solver.NumConstraints())

### Step 5
After declaring the **Objective function** and **Constraints** to the ortools library, we can now invoke the solver to find the **optimal basic feasible solution** for us.

In [ ]:
# Invoke the LP Solver from ortools with the solver.Solve() function
"""
    solver.Solve()
    
    Description: OR Tools will solve the LP problem based on the variables, Objective and Constraints given to
                 it.
    
    Input arguments:
        None
    returns: 
        A string value that can be compared several flags within the Solver() API in ortools
"""
solver_status = solver.Solve()


### Step 6
Check if the LP solver has managed to find the **optimal Basic Feasible Solution** and display the results \
The Status from the LP Solver can be verified by comparing with flags in the *pywraplp.Solver()* API.

Here is a table showing the status with their corresponding flag number:

| Solver status   | Flag Number    |
| --------------- |:--------------:|
| OPTIMAL         | 0              |
| FEASIBLE        | 1              |
| INFEASIBLE      | 2              |
| UNBOUNDED       | 3              |
| ABNORMAL        | 4              |
| NOT_SOLVED      | 6              |


In [ ]:
# Using the status after invoking solver.Solve(), which is saved into solver_status.
# Compare it to pywraplp.Solver.OPTIMAL flag, which is 0

if solver_status == pywraplp.Solver.OPTIMAL:
    print(f"The flag number for the solver status is {solver_status}")
    print("---------------------------------------------------------")
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
    print("---------------------------------------------------------")
else:
    print(f"The flag number of the LP Solver is {solver_status}")
    print("The problem does not have an optimal solution.")
    

As seen from the results, if the solver is able to find the OPTIMAL basic feasible solution, it can show you the **Objective value (Z-value)**, and the values for the **Optimal BFS**.

### Summary of Steps
1. Import the ortools library and declare the **Solver**.
2. Declare **Variables** in LP problem.
3. Declare the **Objective function**.
4. Declare the **Constraints**.
5. Call the ortools solver to find the **Optimal BFS**.
6. Check the **Solver status** and display the **results**.

***
***

In [ ]:
# Clear OR Tools of all variables, Objective functions and Constraints using the solver.Clear() function
solver.Clear()

# Check if solver is cleared of all declared variables and values
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('Objective value =', solver.Objective().Value())

***
***

## Practice 2: Hands-on
Now, you will have to follow the steps as gone through with Practice 1 and solve the next LP problem.

The next LP problem statement is as follows...

\begin{split}
    x_1 = \mbox{ number of soccer balls produced in a day } \\
    x_2 = \mbox{ number of basketballs produced in a day } \\
    x_3 = \mbox{ number of pingpong balls produced in a day }
\end{split} 


\begin{split}
    \begin{array}{r}
        \max \\ \mbox{s.t.} \\ \\ \\ \\ \\ \\
    \end{array} &
    \begin{array}{rcrcll}
        500x_1 & + & 800x_2 & + & 300x_3 & & & \\ 
        5x_1 & + & 2x_2 & + & 3x_3 & \leq & 300\quad\! & \mbox{(rubber)} \\	
        3x_1 & + & 8x_2 & + & x_3 & \leq & 700\quad\! & \mbox{(cloth)} \\
        2x_1 & + & 5x_2 & + & 4x_3 & \leq & 550\quad\! & \mbox{(cloth)} \\
        35x_1 & + & 12x_2 & + & 28x_3 & \leq & 10500\:\ & \mbox{(machine)} \\
        x_1 \geq  0, & x_2 \geq  0, & x_3 \geq  0
    \end{array}
\end{split}

Based on the LP Problem above, construct the solver by following the steps as shown above.

In [ ]:
### Type in your code here ###

## Step 1: Import Library functions and declare solver


## Step 2: Declare variables


## Step 3: Declare Objective Function
# Initialise the solver.Objective() function from ortools library

# Set Coefficients for the variables with objective.SetCoefficient(args1, args2)

# Sets the optimization direction to maximize


## Step 4: Declare the Constraints

## Step 5: Call solver to find Optimal BFS

## Step 6: Check on solver status and display results


######## End of Code #########

## Model-data Decoupling

As explained in the slides previously, **model-data decoupling** is used as a way to make our program more **flexible** and **extendable**. \

### Recap
- You can use lists to store the data/parameters (OR another common way of making the data customizable is the use configuration files (_*_.yaml files), OR using argument parsers via Command Line Interface if you are using a script. )

- Model (Code Program) only contains the abstract model with minimal **hard-coded parameters/values**. 

### Let's try creating our decoupled model-data solver from Practice 1

\begin{split}
    \begin{array}{r}
        \max \\ \mbox{s.t.} \\ \\ \\ \\ \\ 
    \end{array} &
    \begin{array}{rcrcll}
        700x_1 & + & 900x_2 & & & \\ 
        3x_1 & + & 5x_2 & \leq & 3600\quad\! & \mbox{(wood)} \\		
        x_1 & + & 2x_2 & \leq & 1600\quad\! & \mbox{(labor)} \\		
        50x_1 & + & 20x_2 & \leq & 48000\:\ & \mbox{(machine)} \\
        & & x_1 & \geq & 0
        \\
        & & x_2 & \geq & 0.
    \end{array}
\end{split}


### Creating lists to store data for model

Now we will use the method first explained in the slides. \
Create lists to store the data that would be used by the model/solver.

In [ ]:
products = range(2)   # 2 products
resources = range(3)  # 3 resources

prices = [700, 900]   # Objective function

resource_consumptions = [[3 , 5 ],
                         [1 , 2 ],
                         [50, 20]]
resource_limitations = [3600, 1600, 48000]

# flag for maximization or minimization
# 0 for min, 1 for max
max_or_min = 1

Now, create a structure by following the steps to creating a LP solver, where the model will be able to take in the data from the lists created above.

In [ ]:
# Step 1: Declare solver
decoupled_solver = pywraplp.Solver.CreateSolver('GLOP')

# Step 2: Declare variables from 'products' list and save variables into list x
x = []
for i in products:
    x.append(decoupled_solver.NumVar(0, decoupled_solver.infinity(), name = 'x' + str(i+1)))
print("Number of variables : ", solver.NumVariables())

# Step 3: Declare Objective Function
obj_func = decoupled_solver.Objective()
if max_or_min == 0:   # Minimization of objective
    for i in products:
        obj_func.SetCoefficient(x[i], prices[i])
    obj_func.SetMinimization()
else:                 # Maximization of objective
    for i in products:
        obj_func.SetCoefficient(x[i], prices[i])
    obj_func.SetMaximization()

# Step 4: Declare the Constraints
for j in resources:
    decoupled_solver.Add((sum(resource_consumptions[j][i] * x[i] for i in products) 
                           <= resource_limitations[j]))
print("Number of Constraints: ", solver.NumConstraints())

# Step 5: Call solver to find Optimal BFS
solver_status = decoupled_solver.Solve()

# Step 6: Check on solver status and display results
if solver_status == pywraplp.Solver.OPTIMAL:
    print(f"The flag number for the solver status is {solver_status}")
    print("---------------------------------------------------------")
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for i in products:
        print(f"Solution value for {x[i]} is : {x[i].solution_value()}")
else:
    print(f"The problem does not have an optimal solution with Flag number {solver_status}.")


----
Here is a video where you can see more examples on formulating the LP problem into one that can be solved by code

- [youtube link](https://www.youtube.com/watch?v=zZAobExOMB0&ab_channel=NDCConferences)